In [1]:
import pandas as pd
import os
import chardet
from functools import reduce

def select_cols(df, cols=None):
    '''
    returns the selected list of columns, all of them if None
    accepts a list, str or None
    '''
    if cols == None:
        col_list = df.columns
    elif type(cols) == str:
        col_list = [cols]
    else:
        col_list = cols
    return col_list

def delete_char(df, char='', cols=None):
    '''
    deletes a given char, or string from the supplied columns
    if no columns passed, applies to all columns
    '''
    df = df.copy()
    col_list = select_cols(df, cols)
        
    for col in col_list:
        df[col] = df[col].str.replace(char,'')
        
    return df

def convert_to_numeric(df, cols=None, errors='coerce'):
    '''
    converts the given columns to numeric
    useful when there were special characters eliminated before, leaving the data type as string
    '''
    df = df.copy()
    col_list = select_cols(df, cols)
        
    for col in col_list:
        df[col] = pd.to_numeric(df[col], errors=errors)
        
    return df

def convert_to_int(df, cols=None):
    '''
    converts the given columns to int
    '''
    df = df.copy()
    col_list = select_cols(df, cols)
        
    for col in col_list:
        df[col] = df[col].apply(lambda x: int(x))
        
    return df

# a bit local to this task, but works
def read_multiple_csv(folder='.', index_col='id'):
    data_array = []
    filename_list = os.listdir(folder)    
    for filename in filename_list:
        data = pd.read_csv('{}/{}'.format(folder, filename))
        data_array.append(data)
    return pd.concat(data_array, ignore_index=True)

def apply_any_function(df, function=(lambda x: x),  cols=None):
    df = df.copy()
    col_list = select_cols(df, cols)
        
    for col in col_list:
        df[col] = df[col].apply(function)
        
    return df

In [2]:
dataA = pd.read_csv('data/dataA.csv', na_values='?')
dataB = pd.read_csv('data/dataB', sep='\t', na_values='cute_cat').drop(99999).pipe(convert_to_int, 'SK_ID_CURR')
dataB_good = pd.read_csv('data/dataB_good', sep='\t') # is the same as dataB
dataC_1 = pd.read_excel('data/dataC.xlsx', sheet_name=0, header=1)
dataC_2 = pd.read_excel('data/dataC.xlsx', sheet_name=1)
dataC_3 = pd.read_excel('data/dataC.xlsx', sheet_name=2, header=2)

dataC = ( pd.concat(
          [dataC_1.merge(dataC_2, on='SK_ID_CURR'), 
           dataC_3.pipe(convert_to_numeric, 'SK_ID_CURR').dropna(how='all').pipe(convert_to_int, 'SK_ID_CURR')]
          , ignore_index=True)
          .pipe(apply_any_function, lambda x: 1 if x in ('Yay', 1) else 0, 'FLAG_DOCUMENT_19')
        )

dataD = (pd.read_csv('data/dataD.csv', encoding='ISO-8859-1')
        .pipe(apply_any_function, function=(lambda x: x if x in ('Unaccompanied', 'Children', 'Family', 'Spouse, partner',
           'Group of people') else pd.np.nan), cols='NAME_TYPE_SUITE')
        .pipe(apply_any_function, lambda x: 1 if x=='Y' else 0, 'FLAG_OWN_REALTY')
        )

dataD_1 = pd.read_csv('data/dataD_1.csv', encoding='ISO-8859-1')

dataE = pd.concat([pd.read_csv('data/dataE-1.tmp', na_values=' '),
                    pd.read_csv('data/dataE-2.tmp', na_values=' ')]
          , ignore_index=True
         ).set_index('SK_ID_CURR').dropna(how='all').reset_index()

dataF = (pd.read_csv('data/dataF.csv')
            .drop(columns=['REGION_RATING_CLIENT_W_CITY.1', 
                        'EXT_SOURCE_1.1', 
                        'DAYS_REGISTRATION.1', 
                        'EXT_SOURCE_3.1',
                        'LIVINGAREA_MEDI.1',
                        'HOUR_APPR_PROCESS_START.1',
                        'FLOORSMIN_MEDI.1',
                        'FLAG_DOCUMENT_14.1', 
                        'OCCUPATION_TYPE.1', 
                        'LANDAREA_MEDI.1'
                       ]
                 )
             .rename({'sk_id_curr':'SK_ID_CURR'}, axis=1)
        )
dataG = read_multiple_csv('data/dataG').pipe(apply_any_function, lambda x: pd.to_numeric(x.replace('€', '')), 'AMT_GOODS_PRICE')

dataZ = (pd.read_json('data/dataZ_best.csv')
         .pipe(delete_char, '%', 'EXT_SOURCE_2').pipe(convert_to_numeric, 'EXT_SOURCE_2')
         .pipe(apply_any_function, lambda x: 2 if type(x)==str else x)
         .pipe(apply_any_function, lambda x: pd.np.nan if x == -99999 else x, 'LIVINGAREA_AVG')
         .reset_index()
         .rename({'index': 'SK_ID_CURR'}, axis=1)
         ) 

data_list = [dataA, dataB, dataC, dataD, dataE, dataF, dataG, dataZ]

data = reduce(lambda left,right: pd.merge(left,right), data_list)

In [4]:
data.to_csv('data_files.csv')